## Helper functions

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import numpy as np

from others.implementations import *
import torch
from target_model import *
from torch import optim

torch.set_default_tensor_type(torch.DoubleTensor)


In [53]:
def generate_model(shallow_hidden_channels, deep_hidden_channels):
    return  nn.Sequential(
            nn.Conv2d(3, shallow_hidden_channels, kernel_size=2, stride=2, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv2d(shallow_hidden_channels, deep_hidden_channels, kernel_size=2, stride=2, padding=0, bias=False),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(deep_hidden_channels, shallow_hidden_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(shallow_hidden_channels, 3, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Sigmoid()
            )

def test_and_train(batch_size, hidden_channels, lr, momentum, nesterov, epochs):
    network = generate_model(shallow_hidden_channels=hidden_channels[0], deep_hidden_channels=hidden_channels[1])
    optimizer = optim.SGD(network.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)

    # Instantiate model and replace network and optimizer
    m = Model()
    m.model = network
    m.optimizer = optimizer
    m.batch_size = batch_size

    # Train
    m.train(s1, s2, epochs)

    return compute_psnr(m.predict(t1)/255.0, t2), m

def sample(tensor1, tensor2, k):
    perm = torch.randperm(tensor1.size(0))
    idx = perm[:k]
    return tensor1[idx], tensor2[idx]

## Loading data

In [54]:
path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'
noisy_imgs_1, noisy_imgs_2 = torch.load(path_train)
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.double(), noisy_imgs_2.double() / 255.0
test, truth = torch.load(path_val)
test, truth = test.double(), truth.double() / 255.0

## Parameters to tune

In [55]:
# Channels parameter
shallow_hidden_channels = [8, 16]
deep_hidden_channels = [16, 32]
hidden_channels = [(shallow_channel, deep_channel) for shallow_channel in shallow_hidden_channels for deep_channel in deep_hidden_channels if shallow_channel <= deep_channel]

# Optimizer parameters
lrs = np.logspace(-7, -1, 5)
momentums = [0.9]
nesterovs = [False]

# Batch sizes
batch_sizes = [5, 100]

## Training

In [56]:
s1, s2 = sample(noisy_imgs_1, noisy_imgs_2, 5000)
t1, t2 = sample(test, truth, 5000)

In [57]:
epochs = 5

results = dict()
for lr in lrs:
    for momentum in momentums:
        for nesterov in nesterovs:
            for hidden_channel in hidden_channels:
                for batch_size in batch_sizes:
                    description = f'lr{lr}_HiddenChannels{hidden_channel}_Batch{batch_size}_Epochs{epochs}_Momentum{momentum}_Nesterov{nesterov}_Sample1000'
                    print("\nSTARTING TRAINING FOR:", description)
                    error, m = test_and_train(batch_size, hidden_channel, lr, momentum, nesterov, epochs)
                    description = f'lr{lr}_HiddenChannels{hidden_channel}_Batch{batch_size}_Epochs{epochs}_Momentum{momentum}_Nesterov{nesterov}_Sample1000_{error}'
                    results[description] = m
                    print(f'PSNR: {error}')


STARTING TRAINING FOR: lr1e-07_HiddenChannels(8, 16)_Batch5_Epochs5_Momentum0.9_NesterovFalse_Sample1000
EPOCH 0 --- LOSS 0.5664683229291773
EPOCH 1 --- LOSS 0.5507991667674548
EPOCH 2 --- LOSS 0.5319034412923924
EPOCH 3 --- LOSS 0.5098691099228326
EPOCH 4 --- LOSS 0.48599205754128927
FINAL LOSS 0.46282220439511507
PSNR: 3.342843429656214

STARTING TRAINING FOR: lr1e-07_HiddenChannels(8, 16)_Batch100_Epochs5_Momentum0.9_NesterovFalse_Sample1000
EPOCH 0 --- LOSS 0.4315598685758175
EPOCH 1 --- LOSS 0.4310625059569156
EPOCH 2 --- LOSS 0.4304558366768842
EPOCH 3 --- LOSS 0.42984729267425825


KeyboardInterrupt: 

## Final results